In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import base64
from pymongo import MongoClient
from bson.json_util import dumps

from CRUD import AnimalShelter


#############################
# Data Manipulation / Model #
#############################
username = "ryanrieth"             # Hard-coded to ensure proper and successful development
password = "SNHUMongoDB77"         # Hard-coded to ensure proper and successful development

shelter = AnimalShelter(username, password)

# ReadAll() is utilized to further highlight function's purpose of reading the entire CSV file
df = pd.DataFrame.from_records(shelter.readAll({}))


###########################
# Dashboard Layout / View #
###########################
app = JupyterDash('ProjectTwo')

image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Img(
            
            # Grazioso Salvare logo structured in the top left of the dashboard for professional appearance
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'height'        : '20%',
                'width'         : '20%',
                'float'         : 'left',
                'position'      : 'relative',
                'padding-top'   : 0,
                'padding-right' : 0
            })
    ], href='https:/www.github.com/ThatGreaserGuy'),
    html.Br(),
    html.Hr(),
    
    # Unique identifier structured under professional title
    html.Center(html.B(html.H1('Search and Rescue Candidate Locator'))),
    html.Center(html.B(html.H2('Grazioso Salvare'))),
    html.Center(html.B(html.H3('Ryan Rieth - CS 340 - Prof. Kellogg - SNHU'))),
    html.Hr(),
    html.Div(
        
        # Buttons to press that filters the information based upon the button's title
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label' : 'Water Rescue', 'value': 'WR'},
                {'label' : 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                {'label' : 'Disaster/Individual Tracking', 'value': 'DIT'},
                {'label' : 'Reset', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        
        # Only one row should be selected at a time to work in tandem with map updating
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=15,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Br(),
    html.Hr(),
    html.Center(html.Header("Ryan Rieth - CS 340"))
])


###############################################
# Interaction Between Components / Controller #
###############################################

# In the following method, the ages, breeds, and sex of the animals are specifically requested by
# Grazioso Salvare, and so they are specified with AND and OR tools to allow minute filtering
# pertinent to Grazioso Salvare's needs
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    # Update the dashboard's information to show all animals
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
    # Update the dashboard's information to show Water Rescue animals
    elif filter_type == 'WR':
        df = pd.DataFrame.from_records(list(shelter.readAll({'$and': [{'sex_upon_outcome':'Intact Female'},
                                                            {'$or': [
                                                                {'breed':'Labrador Retriever Mix'},
                                                                {'breed':'Chesa Bay Retr Mix'},
                                                                {'breed':'Newfoundland Mix'}]
                                                            },
                                                            {'$and': [{'age_upon_outcome_in_weeks': {'$gte':26}},
                                                                      {'age_upon_outcome_in_weeks': {'$lte':156}}]
                                                            }]})))
        
    # Update the dashboard's information to show Moutain/Wilderness Rescue animals
    elif filter_type == 'MWR':
        df = pd.DataFrame.from_records(list(shelter.readAll({'$and': [{'sex_upon_outcome':'Intact Male'},
                                                            {'$or': [
                                                                {'breed':'Alaskan Malamute'},
                                                                {'breed':'German Shepherd'},
                                                                {'breed':'Old English Sheepdog'},
                                                                {'breed':'Rottweiler'},
                                                                {'breed':'Siberian Husky'}]
                                                            },
                                                            {'$and': [{'age_upon_outcome_in_weeks': {'$gte':26}},
                                                                      {'age_upon_outcome_in_weeks': {'$lte':156}}]
                                                            }]})))
        
    # Update the dashboard's information to show Disaster/Individual Tracking animals
    elif filter_type == 'DIT':
        df = pd.DataFrame.from_records(list(shelter.readAll({'$and': [{'sex_upon_outcome':'Intact Male'},
                                                            {'$or': [
                                                                {'breed':'Bloodhound'},
                                                                {'breed':'Doberman Pinscher'},
                                                                {'breed':'German Shepherd'},
                                                                {'breed':'Golden Retriever'},
                                                                {'breed':'Rottweiler'}]
                                                            },
                                                            {'$and': [{'age_upon_outcome_in_weeks': {'$gte':20}},
                                                                      {'age_upon_outcome_in_weeks': {'$lte':300}}]
                                                            }]})))
        
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data,columns)

# - - - - - - - - - - - - - - - - - - - - - - #

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# - - - - - - - - - - - - - - - - - - - - - - #

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return [
        dcc.Graph(
            
            # Personal favorite graph (pie graph) chosen to portray information in visual form
            figure = px.pie(
                data_frame = dff,
                values = values,
                names = names,
                color_discrete_sequence = px.colors.sequential.RdBu,
                width = 800,
                height = 500
            )
        )
    ]

# - - - - - - - - - - - - - - - - - - - - - - #
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
    dff = pd.DataFrame.from_dict(viewData)
    
    # Default map information to first row's animal in filtered/unfiltered list of animals
    # if no animal's row is manually selected
    if selected_rows == []:
        selected_rows = [0]
    
    # Change animal's lead signifying information from its name to its kind if no name
    # was provided in the CSV file
    if dff.iloc[selected_rows[0],9] is "":
        header = "Animal Type"
        body = dff.iloc[selected_rows[0],3]
    else:
        header = "Animal Name"
        body = dff.iloc[selected_rows[0],9]
    
    # Fillout map's tooltip and pop-up information which flows from the selected row in the
    # datatable widget
    return [
        dl.Map(style={'width':'1000px', 'height':'500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]], children=[
                dl.Tooltip(dff.iloc[selected_rows[0],4]),
                dl.Popup([
                    html.H3(header),
                    html.P(body),
                    html.H3("Sex and Stature"),
                    html.P(dff.iloc[selected_rows[0],12]),
                    html.H3("Age"),
                    html.P(dff.iloc[selected_rows[0],1]),
                    html.H3("ID"),
                    html.P(dff.iloc[selected_rows[0],2])
                ])
            ])
        ])
    ]
    
    
app